## Baseball Analytics With SQLite

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import os

## Getting started

Download the [Lahman's baseball database](http://www.seanlahman.com/baseball-archive/statistics/) from Canvas. 
- You will need to unzip the file and include it in the same directory as your notebook. 
- You will have to name your unzipped file `lahmansbaseballdb.sqlite` 
- If you are not quite sure what to do for this step, please consult the section on the file system in the INFO Paratechnical Handbook which has additional information. Then please post to Canvas for help!
- Note the database is current only to 2018. 

In [2]:
# here is how you connect to the Lahman database and 
# describe its tables

import sqlite3

 # pass a string pointing to the .sqlite file on your machine
con = sqlite3.connect("lahmansbaseballdb.sqlite")

# get the db name
db_name = pd.read_sql("PRAGMA database_list;", con)["name"][0]
db_name

'main'

In [3]:
# Let's list the first five tables in the database

list_tables = "SELECT * FROM {}.sqlite_master WHERE type='table';".format(db_name)

lahmans = pd.read_sql(list_tables , con=con)

lahmans[0:5]

,type,name,tbl_name,rootpage,sql
0,table,allstarfull,allstarfull,2,"CREATE TABLE ""allstarfull"" (\n\t""ID"" INTEGER N..."
1,table,appearances,appearances,151,"CREATE TABLE ""appearances"" (\n\t""ID"" INTEGER N..."
2,table,awardsmanagers,awardsmanagers,3436,"CREATE TABLE ""awardsmanagers"" (\n\t""ID"" INTEGE..."
3,table,awardsplayers,awardsplayers,3445,"CREATE TABLE ""awardsplayers"" (\n\t""ID"" INTEGER..."
4,table,awardssharemanagers,awardssharemanagers,3603,"CREATE TABLE ""awardssharemanagers"" (\n\t""ID"" I..."


In [4]:
# data is only current to 2018! This is important as you draw
# conclusions from the data
pd.read_sql_query('select max(yearid) from halloffame', con) 


,max(yearid)
0,2018


## Deliverable 1

The database contains a table called "halloffame". Use a SELECT statement to print out the first five records of the halloffame table. Use a [limit clause](https://www.sqlitetutorial.net/sqlite-limit/) to only select the first five rows of the table. Note that you will need to use the pandas `read_sql_query` method which takes a connection arguments stored in the variable `con`. 

In [5]:
my_sql_statement = None 

pd.read_sql_query('select*from halloffame limit 5', con)


,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,1,cobbty01,1936,BBWAA,226,170,222,Y,Player,None
1,2,ruthba01,1936,BBWAA,226,170,215,Y,Player,None
2,3,wagneho01,1936,BBWAA,226,170,215,Y,Player,None
3,4,mathech01,1936,BBWAA,226,170,205,Y,Player,None
4,5,johnswa01,1936,BBWAA,226,170,189,Y,Player,None


## Deliverable 2

What are the fields in the hall of fame table? What do you think they represent?

    There is ID, playerID, yearid, votedBy, ballots, needed, votes, inducted, category, and needed_note. I believe 'ID' is for how the players are numbered in the leauge. 'yearid' is the year the player was accepted into the leauge, 'voteBy' is the orginization, ballots is the total ballots for voting players in. 'needed' is how many votes a player needs to be accepted on the team, while 'votes' was the actual votes each player received. 'inducted' is for if the player was inducted into the leauge or not, 'category' is a system descriptor indicating it is a player, and 'needed_note' is simply any notes the owner of the table wants to make.

## Deliverable 3

The `halloffame` table includes a field called `playerid`. Often when you have an `id` field in a database it links to another table in the db. Often working with a database means using SQL to explore and learn the structure of the db. Use the `lahmans` dataframe from above to list the tables in the basebaseball database. Try to discover which table might contain a `playerid` field as a primary key. Then fill out the cell below to select the first five rows from the table. 

In [6]:
list_from_table = 'select playerid from people limit 5' # discovered playerID was in the 'people' table so 
# i am selecting from there and using a limit of 5 for the first 5 rows.
pd.read_sql_query(list_from_table, con)
# lahmans # used to find the list of table names

,playerID
0,aardsda01
1,aaronha01
2,aaronto01
3,aasedo01
4,abadan01


## Deliverable 4

Looking again at the tables in the notebook, notice that there is an `allstarfull` table. Let's look at that table.

In [7]:
pd.read_sql_query('select * from allstarfull', con)

,ID,playerID,yearID,gameNum,gameID,teamID,team_ID,lgID,GP,startingPos
0,1,gomezle01,1933,0,ALS193307060,NYA,921,AL,1,1.0
1,2,ferreri01,1933,0,ALS193307060,BOS,912,AL,1,2.0
2,3,gehrilo01,1933,0,ALS193307060,NYA,921,AL,1,3.0
3,4,gehrich01,1933,0,ALS193307060,DET,919,AL,1,4.0
4,5,dykesji01,1933,0,ALS193307060,CHA,915,AL,1,5.0
...,...,...,...,...,...,...,...,...,...,...
5337,5369,smithwi04,2019,0,ALS201907090,SFN,2920,NL,1,NaN
5338,5370,sorokmi01,2019,0,ALS201907090,ATL,2897,NL,1,NaN
5339,5371,storytr01,2019,0,ALS201907090,COL,2904,NL,1,NaN
5340,5372,woodrbr01,2019,0,ALS201907090,MIL,2911,NL,1,NaN


Looking at the `halloffame` table and the `allstarfull` table we can start to form a question. 

Are there players that were often all stars but who were not elected to the hall of fame? Maybe these are **forgotten stars**! They should have made it to the hall of fame, but they were robbed! 

Notice that this question emerges from the process of exploring the data. This is a very common pattern.

To investigate the question, we need to find out how many all star games each player played in. This will require joins and aggregation. 

But before we are ready to proceed, we need to understand the data a little better. It is **very** common and **very** important to take time to understand data before drawing conclusions. In this case, it seems like the all_star table has a `gameNum` field.

What does that field represent? To investivate you will need to check the documentation or field list for the database in the [readme](http://www.seanlahman.com/files/database/readme2017.txt). Learning how to read and make sense of documentation (including data documentation) is an important skill. 

Another useful way to understand what information is represented in a database is to run SQL queries, such as the one below.

In [8]:
pd.read_sql_query('select distinct gameNum from allstarfull', con)

,gameNum
0,0
1,1
2,2


Based on your investigation, what does `gameNum` describe in the `allstarfull` table?

    gameNum describes how many all star games were played in a given season. In the documentation it states that
    if one allstar game was played that season gameNum = 0, so if 2 were played gameNum = 1, and so forth.

## Deliverable 5

- To start to answer our question, create dataframe called all_stars 
- To create the dataframe, use SQL to join allstarfull with the people table on playerId
- Select the following fields: 
    - 'allstarfull.playerID', 'nameLast', 'nameFirst', 'gameID'
    - 'allstarfull.playerID' specifies that you are taking the playerID field from the `allstarfull` table

In [9]:
myQuery = 'select allstarfull.playerID, nameLast, nameFirst, gameID from allstarfull inner join people on allstarfull.playerID=people.playerID'
all_stars = pd.read_sql_query(myQuery, con)
all_stars

,playerID,nameLast,nameFirst,gameID
0,aaronha01,Aaron,Hank,NLS195507120
1,aaronha01,Aaron,Hank,ALS195607100
2,aaronha01,Aaron,Hank,NLS195707090
3,aaronha01,Aaron,Hank,ALS195807080
4,aaronha01,Aaron,Hank,NLS195907070
...,...,...,...,...
5337,zitoba01,Zito,Barry,ALS200307150
5338,zitoba01,Zito,Barry,NLS200607110
5339,zobribe01,Zobrist,Ben,NLS200907140
5340,zobribe01,Zobrist,Ben,NLS201307160


## Deliverable 6

Pause to check and consider your results. The dataframe `all_stars` says the player aaronha01 played in all star games. Is that right? Well who is `aaronha01`? Use code to check your intuitions about this data by querying the `people` table to learn more about `aaronha01`. Answer by writing a select statement with a where clause.

In [10]:
# delete me answer
your_query = "select * from people where playerid = 'aaronha01' " 
rw = pd.read_sql_query(your_query, con)
rw

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,bats,throws,debut,finalGame,retroID,bbrefID,birth_date,debut_date,finalgame_date,death_date
0,aaronha01,1934,2,5,USA,AL,Mobile,None,None,None,...,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01,1934-02-05,1954-04-13,1976-10-03,None


## Deliverable 7 

Look up the player associated with the ID `aaronha01` on Wikipedia.  Who is aaronha01?

    aaronha01 is Henry Aaron, a famous baseball player who started out in 1954.

## Deliverable 8

Look at the next cell of code and describe what the three lines of code are doing. You may need to consult the pandas documentation.

- Line 1. groupby counts up the playerID for each player and numbers how many games they played in. It also makes a table of the allstar players and the number of allstar games they played.
- Line 2. Resets the index to the default index
- Line 3. Renames the column using a key as the reference to 'N_all_star_games' 

In [11]:
g = all_stars.groupby('playerID').size() # line 1
all_star_counts = g.reset_index() # line 2
all_star_counts = all_star_counts.rename(columns={0: "N_all_star_games"}) # line 3
all_star_counts
# g

,playerID,N_all_star_games
0,aaronha01,24
1,aasedo01,1
2,abreubo01,2
3,abreujo02,3
4,acunaro01,1
...,...,...
1862,zimmejo02,2
1863,zimmery01,2
1864,ziskri01,2
1865,zitoba01,3


## Deliverable 9

Sort `all_star_counts` on the field `N_all_star_games` (higher number of `N_all_star_games` should come first). Which players played in the most all star games? (You may need to join on the people table to get full names).

In [18]:
all_star_counts.sort_values("N_all_star_games", ascending = False)

,playerID,N_all_star_games
0,aaronha01,24
1079,mayswi01,24
1204,musiast01,24
1033,mantlmi01,19
1416,ripkeca01,19
...,...,...
1136,millesh01,1
1138,millwke01,1
1139,milnaal01,1
1140,miltoer01,1


In [16]:
myQuery = 'select allstarfull.playerID, nameLast, nameFirst, gameID from allstarfull inner join people on allstarfull.playerID=people.playerID'
all_stars = pd.read_sql_query(myQuery, con)
all_stars
#theTop = "select N_all_star_games), playerID from all_stars group by N_all_star_games order by(N_all_star_games) desc" 
# topNames = pd.read_sql_query(theTop, con)
# topNames
# top = "select max(N_all_star_games) from all_star_counts"
# topNames = pd.read_sql_query(top, con)
# topNames

,playerID,nameLast,nameFirst,gameID
0,aaronha01,Aaron,Hank,NLS195507120
1,aaronha01,Aaron,Hank,ALS195607100
2,aaronha01,Aaron,Hank,NLS195707090
3,aaronha01,Aaron,Hank,ALS195807080
4,aaronha01,Aaron,Hank,NLS195907070
...,...,...,...,...
5337,zitoba01,Zito,Barry,ALS200307150
5338,zitoba01,Zito,Barry,NLS200607110
5339,zobribe01,Zobrist,Ben,NLS200907140
5340,zobribe01,Zobrist,Ben,NLS201307160


In [ ]:
all_star_counts.merge(all_stars, on = "")

## Merging hall of fame data

Now let's look at data from the hall of fame table. Eventually, we will want to join the hall of fame table with the allstarfull table. But for now let's just take a look.

In [14]:
hall = pd.read_sql_query('select * from halloffame', con)
hall

,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None
1,2,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
2,3,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
3,4,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,None
4,5,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,None
...,...,...,...,...,...,...,...,...,...,...
4186,4187,lidgebr01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4187,4188,millwke01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4188,4189,zambrca01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4189,4190,morrija02,2018,Veterans,NaN,NaN,NaN,Y,Player,None


## Deliverable 10

Is each player listed one time in the halloffame table? You will need to investigate the dataframe with pandas (Hint: use `value_counts`).

In [121]:
# yes players are listed multiple times. I used value_counts to count how many times playerID was in the table.
hall.value_counts('playerID')

playerID
roushed01    20
youngro01    19
ruffire01    18
schalra01    18
vanceda01    17
             ..
houttar01     1
howarfr01     1
howear01      1
howelro02     1
ziskri01      1
Length: 1279, dtype: int64

## Deliverable 11

How many times is the player `vanceda01` listed in the `halloffame` table? Why do you think that is? (Hint: look at the "inducted" field in the database documentation). Answer with code using the cell below. Note that a baseball player may be voted to be included in the hall of fame multiple times before they are accepted.

In [125]:
# Vanceda01 was named 17 times based on the table below. I believe it is becuase he was nominated to be inducted
# in the hall of fame 16 times before finally getting in during his last year in 1955. This further proved by 
# the number of votes he received each year receiving 205 in his last year.
# hall.value_counts('inducted')
vanceda = "select * from halloffame where playerID = 'vanceda01'"
counter = pd.read_sql_query(vanceda, con)
counter

,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,104,vanceda01,1936,BBWAA,226,170.0,1,N,Player,None
1,146,vanceda01,1937,BBWAA,201,151.0,10,N,Player,None
2,265,vanceda01,1938,BBWAA,262,197.0,10,N,Player,None
3,381,vanceda01,1939,BBWAA,274,206.0,15,N,Player,None
4,492,vanceda01,1942,BBWAA,233,175.0,37,N,Player,None
5,569,vanceda01,1945,BBWAA,247,186.0,18,N,Player,None
6,686,vanceda01,1946,Nominating Vote,202,NaN,31,N,Player,Top 20
7,764,vanceda01,1947,BBWAA,161,121.0,50,N,Player,None
8,804,vanceda01,1948,BBWAA,121,91.0,23,N,Player,None
9,921,vanceda01,1949,BBWAA,153,115.0,33,N,Player,None


Adjust the `hall_sql_query` to include a where clause to only select rows from the `halloffame` table where inducted is "Y". Store your answer in the variable `inducted_query` below. The next cell should create a dataframe called `hall`. 

In [128]:
inducted_query = hall
hall = pd.read_sql_query("select * from halloffame where inducted = 'Y'", con)
inducted_query

,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None
1,2,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
2,3,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
3,4,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,None
4,5,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,None
...,...,...,...,...,...,...,...,...,...,...
318,4158,guerrvl01,2018,BBWAA,422.0,317.0,392.0,Y,Player,None
319,4159,thomeji01,2018,BBWAA,422.0,317.0,379.0,Y,Player,None
320,4160,hoffmtr01,2018,BBWAA,422.0,317.0,337.0,Y,Player,None
321,4190,morrija02,2018,Veterans,NaN,NaN,NaN,Y,Player,None


Now let's merge the `hall` dataframe with the `all_star_counts` dataframe. 

In [129]:
# I will use pandas to do the join. You could also do this in SQL. The API is similar. 
merged = pd.merge(all_star_counts, hall, how='left', on='playerID')
all_stars_vs_hall = merged[['playerID', "N_all_star_games", "inducted"]]
all_stars_vs_hall = all_stars_vs_hall.fillna(value="N")

## Deliverable 12
Sort the `all_stars_vs_hall` table by `N_all_star_games` in descending order.

In [138]:
games = all_stars_vs_hall.sort_values(by = 'N_all_star_games', ascending = False)
games

,playerID,N_all_star_games,inducted
0,aaronha01,24,Y
1079,mayswi01,24,Y
1204,musiast01,24,Y
1033,mantlmi01,19,Y
1416,ripkeca01,19,Y
...,...,...,...
1136,millesh01,1,N
1138,millwke01,1,N
1139,milnaal01,1,N
1140,miltoer01,1,N


## Deliverable 13

Make a table `missing_hall_of_famers` that shows the 10 players who played in the most all star games who were none the less NOT inducted into the hall of fame. You can filter the `all_stars_vs_hall` table for this, and call the `.head()` method on the filtered dataframe.

In [149]:
missing_hall_of_famers = all_stars_vs_hall["inducted"] == "N"

missingGames = all_stars_vs_hall[missing_hall_of_famers] 
missingGames.sort_values(by = 'N_all_star_games', ascending = False).head(10)

,playerID,N_all_star_games,inducted
1454,rosepe01,17,N
147,bondsba01,14,N
827,jeterde01,14,N
1432,rodrial01,14,N
1418,riverma01,13,N
1105,mcgwima01,12,N
1371,ramirma02,12,N
349,crandde01,11,N
233,cabremi01,11,N
304,clemero02,11,N


## Deliverable 14

Who are the players in your `missing_hall_of_famers` table? Try looking up a few of the players online (e.g. Wikipedia). Does your table make sense? Note: you may need to join the people table to see the players' full names. 

The table makes sense for the first two, Pete Rose played in 17 all star games but was never inducted into the hall of fame. Even though he had a .303 batting average, won 3 world series, and play in 17 games he wasnt inducted. Barry Bonds is also on the list as playing in 14 all star games but wasn't inducted because he was caught taking steroids. however, the third, Derek Jeter was inducted into the hall of fame in 2020, so this table isn't accurate.